<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Generative_Art_with_GANs_(Generative_Adversarial_Networks)_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
from torchvision.utils import save_image
import os

# Define Generator and Discriminator models
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

# Hyperparameters
latent_dim = 100
batch_size = 64
learning_rate = 0.0002
num_epochs = 10000

# Initialize models
generator = Generator(latent_dim)
discriminator = Discriminator()

# Loss function and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Create directory if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')

# Training loop
for epoch in range(num_epochs):
    for _ in range(batch_size):

        # Train Discriminator
        optimizer_D.zero_grad()

        # Real images
        real_imgs = torch.randn(batch_size, 1024)  # Replace this with actual data
        real_labels = torch.ones(batch_size, 1)
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)

        # Fake images
        z = torch.randn(batch_size, latent_dim)
        fake_imgs = generator(z)
        fake_labels = torch.zeros(batch_size, 1)
        fake_loss = adversarial_loss(discriminator(fake_imgs.detach()), fake_labels)

        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        generated_labels = torch.ones(batch_size, 1)  # Try to fool the discriminator
        g_loss = adversarial_loss(discriminator(fake_imgs), generated_labels)
        g_loss.backward()
        optimizer_G.step()

    # Save images and print losses every 100 epochs
    if epoch % 100 == 0:
        save_image(fake_imgs.view(fake_imgs.size(0), 1, 32, 32), f'images/{epoch}.png')
        print(f"Epoch {epoch}/{num_epochs} - D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")